In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import sys
import numpy as np
import glob
import torch

sys.path.insert(1,"/home/showalte/research/prob_seq_queries/")
from seq_queries.utils import read_pkl, write_pkl

In [28]:
df = pd.read_csv("../data/extracts/wikitext_11-13_15_1000mc_500q.csv")

In [29]:
entropy = pd.read_csv('../data/extracts/vocab_mean_entropy_wikitext.csv')

In [30]:
df.hybrid_num_mc_samples.drop_duplicates()

0         10
500      100
1000    1000
Name: hybrid_num_mc_samples, dtype: int64

In [31]:
df = df[df.hybrid_num_mc_samples == 1000]
df.head()

,sequence_id,num_steps,excluded_term,is_gt,ground_truth,random_sampling_est,importance_sampling_est,hybrid_sampling_est,beam_search_lb,random_num_mc_samples,...,beam_search_true_coverage,beam_search_restricted_coverage,beam_search_num_beams,is_top_k,is_top_p,hybrid_min_variance,hybrid_min_var_reduction,hist_len,total_seq_len,dataset_name
1000,0,4,837,0,0.000675,3.130372e-07,0.000645,0.000566,8.198016e-05,669,...,0.106032,0.106620,669,0,0,1.0,0.1,11,15,wikitext
1001,1,4,29,0,0.000001,9.595782e-11,0.000001,0.000003,2.306062e-07,616,...,0.399728,0.399729,616,0,0,1.0,0.1,11,15,wikitext
1002,2,4,389,0,0.000814,3.592877e-07,0.000914,0.000528,4.358864e-05,729,...,0.033704,0.033807,729,0,0,1.0,0.1,11,15,wikitext
1003,3,4,36289,0,0.000660,7.025756e-09,0.000392,0.000337,1.515005e-05,725,...,0.060856,0.061597,725,0,0,1.0,0.1,11,15,wikitext
1004,4,4,1279,0,0.051403,2.865888e-06,0.047209,0.045376,4.289309e-03,617,...,0.079373,0.103527,617,0,0,1.0,0.1,11,15,wikitext


In [32]:
items = ['num_steps','importance_sampling_error','beam_search_error','random_sampling_error']
df['random_sampling_error'] = (np.abs(df.ground_truth - df.random_sampling_est)/df.ground_truth)*100
df['importance_sampling_error'] = (np.abs(df.ground_truth - df.importance_sampling_est)/df.ground_truth)*100
df['beam_search_error'] = (np.abs(df.ground_truth - df.beam_search_lb)/df.ground_truth)*100
ndf = df[items]


In [33]:
res = ndf.groupby(by='num_steps').median()

In [34]:
res

,importance_sampling_error,beam_search_error,random_sampling_error
num_steps,,,
2,11.414524,51.246947,99.355714
3,13.345708,82.422643,99.948371
4,13.531793,93.586815,99.990006


In [35]:
ment = entropy[['num_steps','entropy','lim_entropy']]
ment['rel_entropy'] = (ment.entropy/ment.lim_entropy)*100
ment

,num_steps,entropy,lim_entropy,rel_entropy
0,1,6.725350,15.617037,43.064187
1,2,12.893677,31.234074,41.280805
2,3,19.088594,46.851111,40.743098
3,4,25.226220,62.468148,40.382532
4,5,31.254194,78.085185,40.025767


In [36]:
res = res.merge(ment, how='left',on='num_steps')

In [37]:
res

,num_steps,importance_sampling_error,beam_search_error,random_sampling_error,entropy,lim_entropy,rel_entropy
0,2,11.414524,51.246947,99.355714,12.893677,31.234074,41.280805
1,3,13.345708,82.422643,99.948371,19.088594,46.851111,40.743098
2,4,13.531793,93.586815,99.990006,25.226220,62.468148,40.382532


In [38]:
res.drop('lim_entropy',inplace=True, axis=1)

In [39]:
res.shape

(3, 6)

In [40]:
res

,num_steps,importance_sampling_error,beam_search_error,random_sampling_error,entropy,rel_entropy
0,2,11.414524,51.246947,99.355714,12.893677,41.280805
1,3,13.345708,82.422643,99.948371,19.088594,40.743098
2,4,13.531793,93.586815,99.990006,25.226220,40.382532


In [41]:
for col in res.columns:
    res[col] = np.round(res[col],2)


In [42]:
res

,num_steps,importance_sampling_error,beam_search_error,random_sampling_error,entropy,rel_entropy
0,2,11.41,51.25,99.36,12.89,41.28
1,3,13.35,82.42,99.95,19.09,40.74
2,4,13.53,93.59,99.99,25.23,40.38


In [43]:
res.to_latex("GPT2_res_table_entropy.tex")

<ipython-input-43-1e8157f76721>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  res.to_latex("GPT2_res_table_entropy.tex")
